#Entregable 2:
Detectar automáticamente si las entidades clínicas identificadas en un conjunto de oraciones están afirmadas, negadas o en estado de incertidumbre, mediante un modelo de lenguaje previamente entrenado en detección de negación, afirmación e incertidumbre en textos clínicos.

##Sirve para:

* Clasificar el estado semántico de cada entidad (por ejemplo, "no fiebre" → negación).

* Permite diferenciar entre hechos clínicos confirmados, descartados o dudosos.

##Se puede utilizar en:

* Minería de texto en salud.

* Análisis automatizado de historias clínicas.

* Generación de etiquetas semánticas clínicas que apoyan el diagnóstico, la vigilancia de patologías y el análisis de cohortes.



In [ ]:
#IMPORTAR LIBRERIAS
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, pipeline

In [ ]:
#DEFINICIÓN DE ARGUMENTOS / RUTAS A REPOSITORIOS
#Configuración del modelo
MODEL_NAME = "JuanSolarte99/bert-base-uncased-finetuned-ner-negation_detection_NUBES"

#Configuración del dispositivo
DEVICE = 0 if torch.cuda.is_available() else -1

In [ ]:
#CREACIÓN PIPELINE PARA MODELO NER
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)
negation_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Device set to use cpu


In [ ]:
#Lista de oraciones de validación

oraciones = [
    "Paciente con diagnóstico confirmado de adenocarcinoma pulmonar con metástasis hepáticas activas.",
    "La paciente niega fiebre, pérdida de peso y dolor torácico.",
    "No se evidencian signos de infección respiratoria aguda.",
    "Se sospecha recurrencia tumoral en la región axilar izquierda.",
    "Presenta tos persistente y disnea al esfuerzo desde hace 3 semanas.",
    "Paciente no bebedora y no fumadora desde hace 10 años.",
    "Lesión pulmonar de características atípicas, posible neoplasia maligna.",
    "Recibió quimioterapia con doxorrubicina y ciclofosfamida durante 6 ciclos.",
    "Sin antecedentes familiares de cáncer de mama o de ovario."
    "Quizás haya compromiso pulmonar leve.",
]

In [ ]:
#Aplicar el modelo a cada oración
resultados = []
for oracion in oraciones:
    spans = negation_pipeline(oracion)
    for span in spans:
        resultados.append({
            "sentence": oracion,
            "word": span["word"],
            "label": span["entity_group"]
        })

In [ ]:
#Convertir a DataFrame
df_resultados = pd.DataFrame(resultados)

In [ ]:
#LISTA DE ETIQUETAS ÚNICAS DETECTADAS POR EL MODELO
etiquetas_detectadas = df_resultados["label"].dropna().unique().tolist()
etiquetas_detectadas.sort()

In [ ]:
#IMPRIMIR LISTA DE ETIQUETAS ÚNICAS DETECTADAS POR EL MODELO
print("\nEtiquetas detectadas por el modelo:")
for label in etiquetas_detectadas:
    print(f"- {label}")


Etiquetas detectadas por el modelo:
- LABEL_0
- LABEL_1
- LABEL_2
- LABEL_3
- LABEL_5
- LABEL_6
- LABEL_7
- LABEL_8


In [ ]:
print("Resultados del modelo por oración y palabra:")
print(df_resultados)

Resultados del modelo por oración y palabra:
                                             sentence  \
0   Paciente con diagnóstico confirmado de adenoca...   
1   La paciente niega fiebre, pérdida de peso y do...   
2   La paciente niega fiebre, pérdida de peso y do...   
3   La paciente niega fiebre, pérdida de peso y do...   
4   La paciente niega fiebre, pérdida de peso y do...   
5   La paciente niega fiebre, pérdida de peso y do...   
6   No se evidencian signos de infección respirato...   
7   No se evidencian signos de infección respirato...   
8   No se evidencian signos de infección respirato...   
9   No se evidencian signos de infección respirato...   
10  Se sospecha recurrencia tumoral en la región a...   
11  Se sospecha recurrencia tumoral en la región a...   
12  Se sospecha recurrencia tumoral en la región a...   
13  Se sospecha recurrencia tumoral en la región a...   
14  Se sospecha recurrencia tumoral en la región a...   
15  Se sospecha recurrencia tumoral en la r

In [ ]:
#Interpretación de las etiquetas basada en las oraciones de validación
etiqueta_a_estado = {
    "LABEL_0": "Negación",
    "LABEL_1": "Negación",
    "LABEL_5": "Negación",
    "LABEL_2": "Incertidumbre",
    "LABEL_6": "Incertidumbre",
    "LABEL_3": "Incertidumbre",
    "LABEL_7": "Afirmación",
    "LABEL_8": "Afirmación"
}

In [ ]:
#Detección de estado por oración (prioridad: Negada > Incierta > Afirmada)
def clasificar_estado(grupo_etiquetas):
    if any(e in grupo_etiquetas for e in ["LABEL_0", "LABEL_1"]):
        return "Negación"
    elif any(e in grupo_etiquetas for e in ["LABEL_2", "LABEL_3", "LABEL_6"]):
        return "Incertidumbre"
    else:
        return "Afirmación"

#Asignar estado a cada oración
estados = []

for oracion in df_resultados["sentence"].unique():
    etiquetas = df_resultados[df_resultados["sentence"] == oracion]["label"].unique().tolist()
    estado = clasificar_estado(etiquetas)
    estados.append({"sentence": oracion, "estado_contextual": estado})

#Mostrar resultados finales
df_estados = pd.DataFrame(estados)
print("\n\n Clasificación de contexto por oración:")
print(df_estados)



 Clasificación de contexto por oración:
                                            sentence estado_contextual
0  Paciente con diagnóstico confirmado de adenoca...        Afirmación
1  La paciente niega fiebre, pérdida de peso y do...          Negación
2  No se evidencian signos de infección respirato...          Negación
3  Se sospecha recurrencia tumoral en la región a...     Incertidumbre
4  Presenta tos persistente y disnea al esfuerzo ...        Afirmación
5  Paciente no bebedora y no fumadora desde hace ...          Negación
6  Lesión pulmonar de características atípicas, p...     Incertidumbre
7  Recibió quimioterapia con doxorrubicina y cicl...        Afirmación
8  Sin antecedentes familiares de cáncer de mama ...          Negación
